In [71]:
import pinocchio as pin
import os
from pinocchio.visualize import MeshcatVisualizer
import numpy as np
from numpy.linalg import solve,norm
# from robot_descriptions.loaders.pinocchio import load_robot_description
# import gepetto.corbaserver

In [72]:
urdf_filename = "/home/adi/hum_rob_ws/src/six_dof/urdf/6dof_fixed.urdf"
mesh_dir = "/home/adi/hum_rob_ws/src/six_dof/meshes"

ros_package_path = "/home/adi/hum_rob_ws/src"  # Replace with your path
os.environ["ROS_PACKAGE_PATH"] = ros_package_path

In [73]:
model = pin.buildModelFromUrdf(urdf_filename)
visual_model = pin.buildGeomFromUrdf(
    model, urdf_filename, pin.GeometryType.VISUAL, package_dirs=mesh_dir
)
collision_model = pin.buildGeomFromUrdf(
    model, urdf_filename, pin.GeometryType.COLLISION, package_dirs=mesh_dir
)
data = model.createData()

In [74]:
#list of joitns to lock
joints_to_lock=['right_foot_shin','right_shin_thigh','right_thigh_body','right_thigh_body_lateral']

IDs=[]

for joint_name in joints_to_lock:
    if model.existJointName(joint_name):
        IDs.append(model.getJointId(joint_name))
print(IDs)

initial_joint_config=np.array([0,0,0,0,0,0,0,0])

model_reduced, geom_models_reduced=pin.buildReducedModel(model,list_of_geom_models=[visual_model,collision_model],
    list_of_joints_to_lock=IDs,
    reference_configuration=initial_joint_config)
visual_model_reduced,collision_model_reduced=geom_models_reduced[0],geom_models_reduced[1]

[1, 2, 3, 4]


In [75]:
data_reduced=model_reduced.createData()
geom_data = pin.GeometryData(collision_model_reduced)

In [76]:
print('joints to lock (only ids):', IDs)
print('reduced model: dim=' + str(len(model_reduced.joints)))
print('-' * 30)

joints to lock (only ids): [1, 2, 3, 4]
reduced model: dim=5
------------------------------


In [77]:
visualizer = MeshcatVisualizer(model_reduced, collision_model_reduced, visual_model_reduced)


visualizer.initViewer()
visualizer.loadViewerModel()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7013/static/


In [78]:
q_red=pin.neutral(model_reduced)
print(q_red)
eps = 1e-3
DT = 1e-1
damp = 1e-1
IT_MAX=20000
JOINT_ID = 4

oMdes = pin.SE3(np.eye(3), np.array([0.092, 0.03, 0.05])) #think of oMdes as oTdes, the transfromation from desired to origin expressed in origin
i = 0

visualizer.display(q_red)

q=pin.neutral(model)


[0. 0. 0. 0.]


In [79]:
pin.forwardKinematics(model_reduced,data_reduced,q_red)
 
# Print out the placement of each joint of the kinematic tree
for name, oMi in zip(model_reduced.names, data_reduced.oMi):
    print(("{:<24} : {: .3f} {: .3f} {: .3f}"
          .format( name, *oMi.translation.T.flat )))

print(data_reduced.oMi[4])

universe                 :  0.000  0.000  0.000
body_left_thigh_lateral  :  0.076  0.000  0.290
body_left_thigh          :  0.076  0.000  0.290
left_thigh_shin          :  0.071  0.000  0.130
left_shin_foot           :  0.092 -0.000  0.000
  R =
    0.999995    0.0031853 -2.53654e-06
  -0.0031853     0.999995  4.03983e-09
 2.53654e-06  4.03983e-09            1
  p =       0.0925 -2.70751e-05  2.15606e-08



In [80]:
visualizer.viewer.jupyter_cell()

In [81]:
while True:
    pin.forwardKinematics(model_reduced, data_reduced, q_red)
    
    iMd = data_reduced.oMi[JOINT_ID].actInv(oMdes) #active inverse, chnages basis and moves the vector/point to the appropriate position in the new basis
    err = pin.log(iMd).vector  # in joint frame  The logarithm map (log()) is a mathematical operation that maps elements from the group SE(3) to its associated Lie algebra, se(3). This operation effectively converts the complex rotational and translational transformation into a simpler vector representation that captures the "difference" or "error" between two poses in SE(3).
    norm_err=norm(err)
    if norm_err < eps:
        success = True
        break
    if i >= IT_MAX:
        success = False
        break
    J = pin.computeJointJacobian(
        model_reduced, data_reduced, q_red, JOINT_ID
    )  # jacobian computed in e-e frame
    J = -np.dot(pin.Jlog6(iMd.inverse()), J)
    q_dot_red = -J.T.dot(solve(J.dot(J.T) + damp * np.eye(6), err))
    q_red = pin.integrate(model_reduced, q_red, q_dot_red * DT)
    
    # #Collision check
    # pin.updateGeometryPlacements(
    #     model_reduced, data_reduced, collision_model_reduced, geom_data, q_red
    # )  # updates geom_data by reference

    # # an active pair is a pair of bodies in a joint considered for collision
    # collision_detected = pin.computeCollisions(
    #     model_reduced, data_reduced, collision_model_reduced, geom_data, q_red, True
    # )  # this returns data,geom_data. polymorphic function

    # if not collision_detected:
    #     for idx in range(model_reduced.nq):
    #         # Assuming model.lowerPositionLimit and model.upperPositionLimit store the limits
    #         q_red[idx] = max(
    #             model_reduced.lowerPositionLimit[idx],
    #             min(q_red[idx], model_reduced.upperPositionLimit[idx]),
    #         )
    
    
    visualizer.display(q_red)
    print(norm_err)
    #print(i)
    i+=1
    #time.sleep(0.05)

if success:
    print("Convergence achieved!")
else:
    print(
        "\nWarning: the iterative algorithm has not reached convergence to the desired precision"
    )

print("\nresult: %s" % q_red.flatten().tolist())
print("\nfinal error: %s" % err.T)



0.022369544886408483
0.022334853475442718
0.022306179845852408
0.022282485874595883
0.02226291069429453
0.022246741006296054
0.02223338618236932
0.02222235744117982
0.02221325047296047
0.022205730969042183
0.0221995225895615
0.022194396971516154
0.022190165440065773
0.022186672138814607
0.02218378834030494
0.02218140773679659
0.022179442544366823
0.022177820281183657
0.022176481104189433
0.022175375608022345
0.022174463006368366
0.022173709629584107
0.022173087683783912
0.022172574226019433
0.022172150318012493
0.02217180032739326
0.02217151135077511
0.022171272737450766
0.022171075696181125
0.022170912970592628
0.02217077857122193
0.022170667554327977
0.02217057583931374
0.02217050005801942
0.022170437430327445
0.02217038566148468
0.022170342857352747
0.022170307454456063
0.022170278162244067
0.022170253915435624
0.022170233834684767
0.022170217194115355
0.022170203394524485
0.02217019194126542
0.022170182425992682
0.022170174511594934
0.022170167919758895
0.022170162420704586
0.02217

In [82]:
pin.forwardKinematics(model_reduced,data_reduced,q_red)
 
# Print out the placement of each joint of the kinematic tree
for name, oMi in zip(model_reduced.names, data_reduced.oMi):
    print(("{:<24} : {: .3f} {: .3f} {: .3f}"
          .format( name, *oMi.translation.T.flat )))

universe                 :  0.000  0.000  0.000
body_left_thigh_lateral  :  0.076  0.000  0.290
body_left_thigh          :  0.076  0.000  0.290
left_thigh_shin          :  0.071 -0.052  0.139
left_shin_foot           :  0.093  0.000  0.020
